In [3]:
import pandas as pd
import typing as t
import os
DISPLAY_NAME = "SurpriseNet"
DATASETS = [
    "S-FMNIST", "S-CIFAR10", "S-CIFAR100", "S-CORe50", "SE-CIFAR100", "SE-CORe50"
]
COLUMNS = ["final_accuracy_mean", "final_accuracy_std", "count"]
CSV_FILE = "data/ExperimentLogs.csv"

In [7]:
def load_clean_table():
    exp_data = pd.read_csv(CSV_FILE)

    def repo_version_filter(df: pd.DataFrame, versions: t.Dict[str, t.Set[str]]) -> pd.DataFrame:
        """Filter a dataframe by repo version"""

        def _is_valid(row):
            if row["dataset"] in versions:
                return row["repo_hash"] in versions[row["dataset"]]
            return False
        return df[df.apply(_is_valid, axis=1)]

    def did_complete(row) -> bool:
        """Check if an experiment has completed"""
        return row["completed_tasks"] == row["n_experiences"]

    def remove_unused_parameters(row):
        """Remove unused hyperparameter from rows"""
        if not row["use_packnet"]:
            row["prune_proportion"] = ""
        elif row["prune_proportion"][0] == "[":
            row["prune_proportion"] = "ep"
        if not row["use_experience_replay"] or pd.isna(row["replay_buffer"]):
            row["replay_buffer"] = ""
        return row

    exp_data = repo_version_filter(
            exp_data,
            {
                "S-FMNIST": {"1555acb6", "0a229afa", "4e7023cd", "4e7023cdD", "0a229afaD", "54dcf601"},
                "S-CIFAR10": {"1555acb6", "0a229afa", "4e7023cd", "4e7023cdD", "0a229afaD"},
                "S-CIFAR100": {"1555acb6", "0a229afa", "4e7023cd", "4e7023cdD", "0a229afaD"},
                "S-CORe50":    {"84260321", "a6134f6c", "a3411a3b", "ecc57a24"},
                "SE-CORe50":   {"84260321", "a6134f6c", "a3411a3b", "ecc57a24"},
                "SE-CIFAR100": {"84260321", "a6134f6c", "a3411a3b", "ecc57a24"},
            },
            
    )
    # remove certain experiment categories
    exp_data = exp_data[~exp_data["experiment_category"].isin(["TEST"])]
    exp_data = exp_data[~exp_data["strategy"].isin(["LwF", "SI"])]
    # Filter out incomplete experiments
    exp_data = exp_data[exp_data.apply(did_complete, axis=1)]
    # Simplify hyper-parameters
    exp_data = exp_data.apply(remove_unused_parameters, axis=1)


    group_by_keys = [
        "strategy",
        "architecture",
        "replay_buffer",
        "prune_proportion",
        "dataset"
    ]

    grouped = exp_data.groupby(group_by_keys, dropna=False)

    # Calculate the mean and standard deviation of the final accuracy
    table_components = {
        COLUMNS[0]: grouped["final_accuracy"].mean(),
        COLUMNS[1]: grouped["final_accuracy"].std(),
        COLUMNS[2]: grouped["experiment_code"].count()
    }

    table = pd.DataFrame(
        table_components
    )


    table = table.pivot_table(
            values=COLUMNS, 
            columns="dataset", 
            index=['strategy', 'replay_buffer', 'prune_proportion', 'architecture'])
    return table
clean_table = load_clean_table()
clean_table

count  \
dataset                                                   S-CIFAR10   
strategy      replay_buffer prune_proportion architecture             
finetuning                                   AE                  10   
                                             VAE                 10   
nonContinual                                 AE                  10   
                                             VAE                 10   
replay        100.0                          AE                  12   
              1000.0                         AE                  12   
              10000.0                        AE                  12   
taskInference               0.2              AE                  12   
                                             VAE                 12   
                            0.4              AE                  12   
                                             VAE                 12   
                            0.5              AE                  10   
                                             VAE                 10   
                            0.6              AE                  12   
                                             VAE                 11   
                            0.8              AE                  12   
                                             VAE                 11   
                            ep               AE                  10   
                                             VAE                 10   
taskOracle                  0.5              AE                  10   

                                                                               \
dataset                                                   S-CIFAR100 S-CORe50   
strategy      replay_buffer prune_proportion architecture                       
finetuning                                   AE                   11       11   
                                             VAE                  11       10   
nonContinual                                 AE                   10       11   
                                             VAE                  11       10   
replay        100.0                          AE                   12       10   
              1000.0                         AE                   10       11   
              10000.0                        AE                   10       10   
taskInference               0.2              AE                   10       10   
                                             VAE                  10       10   
                            0.4              AE                   10       10   
                                             VAE                  10       10   
                            0.5              AE                   10       10   
                                             VAE                  10       10   
                            0.6              AE                   10       10   
                                             VAE                  10       10   
                            0.8              AE                   10       10   
                                             VAE                  10       10   
                            ep               AE                   10       11   
                                             VAE                  10       10   
taskOracle                  0.5              AE                   10       11   

                                                                    \
dataset                                                   S-FMNIST   
strategy      replay_buffer prune_proportion architecture            
finetuning                                   AE                 10   
                                             VAE                10   
nonContinual                                 AE                 10   
                                             VAE                10   
replay        100.0                          AE                 14   
    

In [10]:
# Add data from different repositories

from os import PathLike


def add_csv(df: pd.DataFrame, index: t.Tuple, csv_file: PathLike, ):
    data = pd.read_csv(csv_file, index_col=[0], comment="#")

    # Add rows
    for dataset in DATASETS:
        if dataset not in data.columns:
            continue
        df.loc[index, ("final_accuracy_mean",dataset)] = data.mean()[dataset]
        df.loc[index, ("final_accuracy_std",dataset)] = data.std()[dataset]
        df.loc[index, ("count", dataset)] = data.count()[dataset]

    return df
clean_table = add_csv(clean_table, ("SnB", "1000", "", "-"), "data/SnB.csv")
clean_table = add_csv(clean_table, ("BIR", "", "", "VAE"), "data/BIR.csv")
clean_table = add_csv(clean_table, ("GR", "", "", "VAE"), "data/GR.csv")
clean_table = add_csv(clean_table, ("ICARL", "1000", "", "-"), "data/ICARL.csv")
clean_table = add_csv(clean_table, ("taskOracle", "", "Best", "AE or VAE"), "data/BestPackNet.csv")


/tmp/ipykernel_1015861/1392958822.py:13: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[index, ("final_accuracy_mean",dataset)] = data.mean()[dataset]
/tmp/ipykernel_1015861/1392958822.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[index, ("final_accuracy_std",dataset)] = data.std()[dataset]
/tmp/ipykernel_1015861/1392958822.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[index, ("count", dataset)] = data.count()[dataset]
/tmp/ipykernel_1015861/1392958822.py:13: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[index, ("final_accuracy_mean",dataset)] = data.mean()[dataset]
/tmp/ipykernel_1015861/1392958822.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[index, ("final_accuracy_std",dataset)] = data.std()[dataset]
/tmp/ipykernel_1015861/1392958822.py:15: PerformanceWarning: indexing past lexsort depth ma

In [11]:
DISPLAY_STRATEGY_NAMES = {
    "nonContinual": "Non-Continual",
    "taskOracle": "Task Oracle PackNet \cite{Mallya_Lazebnik_2018}",
    "PackNetBest": "PackNet \cite{Mallya_Lazebnik_2018}",
    "replay": "Experience Replay",
    "finetuning": "Finetuning",
    "SnB": "Split and Bridge \cite{Kim_Choi_2021}",
    "GR": "Generative Replay \cite{vandevenBraininspiredReplayContinual2020}",
    "BIR": "Brain Inspired Replay \cite{vandevenBraininspiredReplayContinual2020}",
    "taskInference": f"{DISPLAY_NAME} (ours)",
    "ICARL": "iCaRL \cite{rebuffiICaRLIncrementalClassifier2017}"
}

In [12]:
# Sort by strategy
strategy_order = [
    "nonContinual",
    "taskOracle",
    "PackNetBest",
    "finetuning",
    "replay",
    "SnB",
    "ICARL",
    "GR",
    "BIR",
    "taskInference",
]
clean_table = clean_table.reindex(
    strategy_order,
    level=0,
)
clean_table

count  \
dataset                                                   S-CIFAR10   
strategy      replay_buffer prune_proportion architecture             
nonContinual                                 AE                10.0   
                                             VAE               10.0   
taskOracle                  0.5              AE                10.0   
                            Best             AE or VAE         10.0   
finetuning                                   AE                10.0   
                                             VAE               10.0   
replay        100.0                          AE                12.0   
              1000.0                         AE                12.0   
              10000.0                        AE                12.0   
SnB           1000                           -                 10.0   
ICARL         1000                           -                 10.0   
GR                                           VAE               10.0   
BIR                                          VAE               10.0   
taskInference               0.2              AE                12.0   
                                             VAE               12.0   
                            0.4              AE                12.0   
                                             VAE               12.0   
                            0.5              AE                10.0   
                                             VAE               10.0   
                            0.6              AE                12.0   
                                             VAE               11.0   
                            0.8              AE                12.0   
                                             VAE               11.0   
                            ep               AE                10.0   
                                             VAE               10.0   

                                                                               \
dataset                                                   S-CIFAR100 S-CORe50   
strategy      replay_buffer prune_proportion architecture                       
nonContinual                                 AE                 10.0     11.0   
                                             VAE                11.0     10.0   
taskOracle                  0.5              AE                 10.0     11.0   
                            Best             AE or VAE          10.0     10.0   
finetuning                                   AE                 11.0     11.0   
                                             VAE                11.0     10.0   
replay        100.0                          AE                 12.0     10.0   
              1000.0                         AE                 10.0     11.0   
              10000.0                        AE                 10.0     10.0   
SnB           1000                           -                  10.0     10.0   
ICARL         1000                           -                  10.0     10.0   
GR                                           VAE                10.0     10.0   
BIR                                          VAE                10.0     10.0   
taskInference               0.2              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.4              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.5              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.6              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.8              AE                 10.0     10.0   
                                             VAE

In [14]:
from pandas.io.formats.style import Styler
from IPython.display import HTML
import numpy as np


def create_latex_table(
        df: pd.DataFrame,
        caption: str = "",
        label: str = "",
        bolding_ignores_rows = []) -> str:
    """Convert a dataframe to latex"""

    # Make the index more readable
    df = df.reset_index()
    def _format_hp(row):
        hp = ""
        prune_proportion = row["prune_proportion"][0]
        if prune_proportion == "ep":
            hp = "Equal Prune"
        elif prune_proportion == "Best":
            hp = "Best"
        elif prune_proportion != "":
            hp = f"$\lambda$={float(prune_proportion)*100:.0f}\%"

        replay_buffer = row["replay_buffer"][0]
        if replay_buffer != "":
            hp += f"$n$={int(replay_buffer)}"
        return hp
    # The new index will consist of the strategy, hyper-parameters, and the architecture
    df["Config"] = df.apply(_format_hp, axis=1)
    df["Strategy"]         = df["strategy"].apply(lambda x: DISPLAY_STRATEGY_NAMES[x])
    df["AE/VAE"]     = df["architecture"]
    df = df.drop(columns=["strategy", "replay_buffer", "prune_proportion", "architecture"])
    df.set_index(["Strategy", "Config", "AE/VAE"], inplace=True)
    df = df[[(col, dataset) for col in COLUMNS for dataset in DATASETS]]
    
    
    # Format the values into a format like '81.6±8.6'
    def _format_values(row):
        pretty_row = {}
        for dataset, value in row.groupby("dataset"):
            final_accuracy_mean = value["final_accuracy_mean"][0] * 100
            final_accuracy_std = value["final_accuracy_std"][0] * 100
            if not np.isnan(final_accuracy_mean) and not np.isnan(final_accuracy_std):
                pretty_row[dataset] = f"{final_accuracy_mean:.1f}$\pm${final_accuracy_std:.2f}"
            else:
                pretty_row[dataset] = "-"
            
        return pd.Series(pretty_row)
    
    # Save the indices of the best cells for each dataset
    relevant_rows = np.setdiff1d(list(range(df.shape[0])), bolding_ignores_rows)
    best_rows = df["final_accuracy_mean"].iloc[relevant_rows].idxmax()


    df = df.apply(_format_values, axis=1)
    df = df[DATASETS]
    style: Styler = df.style


    # Bold the best cells
    def _bold_best(row):
        return ["font-weight: bold" if row.name == best_row else "" for best_row in best_rows]
    style = style.apply(_bold_best, axis=1)

    # Export to latex
    result = style.to_latex(
        convert_css=True,
        hrules=True,
        position_float="centering",
        multirow_align="t",
        caption=caption,
        label=label,
    )
    return result

def copy_latex(latex: str):
    latex = latex.replace("\\", "\\\\")
    latex = latex.replace("\n", "\\n")
    return HTML(f"""<button onclick="navigator.clipboard.writeText('""" + latex + """')">COPY</button>""")


In [15]:
df = clean_table.copy(deep=True)
best_idx = df.groupby("strategy").idxmax().loc[("taskInference"), ("final_accuracy_mean")]

# Create new row
idx = ("taskInference", "", "Best", "AE or VAE")
df.loc[idx] = None
row = df.loc[idx]

for key, value in best_idx.items():
    row.loc["final_accuracy_std",key] = clean_table.loc[value].loc["final_accuracy_std",key]
    row.loc["final_accuracy_mean",key] = clean_table.loc[value].loc["final_accuracy_mean",key]
    row.loc["count",key] = clean_table.loc[value].loc["count",key]

df = df.reset_index()
best_vs_rest = df[~((df["strategy"] == "taskInference") & (df["prune_proportion"] != "Best"))]
best_vs_rest.set_index(["strategy", "prune_proportion", "replay_buffer", "architecture"], inplace=True)
best_vs_rest


/tmp/ipykernel_1015861/14884300.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[idx] = None


count  \
dataset                                                   S-CIFAR10   
strategy      prune_proportion replay_buffer architecture             
nonContinual                                 AE                10.0   
                                             VAE               10.0   
taskOracle    0.5                            AE                10.0   
              Best                           AE or VAE         10.0   
finetuning                                   AE                10.0   
                                             VAE               10.0   
replay                         100.0         AE                12.0   
                               1000.0        AE                12.0   
                               10000.0       AE                12.0   
SnB                            1000          -                 10.0   
ICARL                          1000          -                 10.0   
GR                                           VAE               10.0   
BIR                                          VAE               10.0   
taskInference Best                           AE or VAE         11.0   

                                                                               \
dataset                                                   S-CIFAR100 S-CORe50   
strategy      prune_proportion replay_buffer architecture                       
nonContinual                                 AE                 10.0     11.0   
                                             VAE                11.0     10.0   
taskOracle    0.5                            AE                 10.0     11.0   
              Best                           AE or VAE          10.0     10.0   
finetuning                                   AE                 11.0     11.0   
                                             VAE                11.0     10.0   
replay                         100.0         AE                 12.0     10.0   
                               1000.0        AE                 10.0     11.0   
                               10000.0       AE                 10.0     10.0   
SnB                            1000          -                  10.0     10.0   
ICARL                          1000          -                  10.0     10.0   
GR                                           VAE                10.0     10.0   
BIR                                          VAE                10.0     10.0   
taskInference Best                           AE or VAE          10.0     10.0   

                                                                    \
dataset                                                   S-FMNIST   
strategy      prune_proportion replay_buffer architecture            
nonContinual                                 AE               10.0   
                                             VAE              10.0   
taskOracle    0.5                            AE               10.0   
              Best                           AE or VAE        10.0   
finetuning                                   AE               10.0   
                                             VAE              10.0   
replay                         100.0         AE               14.0   
                               1000.0        AE               12.0   
                               10000.0       AE               12.0   
SnB                            1000          -                10.0   
ICARL                          1000          -                10.0   
GR                                           VAE              10.0   
BIR                                          VAE              10.0   
taskInference Best                           AE or VAE        10.0   

                                                                       \
dataset                                                   SE-CIFAR100   
strategy      prune_proportion replay_buffer architecture               
nonContinual                                 AE                  1

In [17]:
latex_table = create_latex_table(
        best_vs_rest,
        caption=f"Best {DISPLAY_NAME} vs Others \\\\ Mean Final Accuracy $\pm$ One Standard Deviation after 10 runs \\\\ Top: Non-Class-IL Methods, Bottom: Class-IL Methods",
        label="tab:best_vs_rest",
        bolding_ignores_rows=[0, 1, 2, 3]
)
print("Best VS Rest")
copy_latex(latex_table)

Best VS Rest


/tmp/ipykernel_1015861/255770649.py:33: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(columns=["strategy", "replay_buffer", "prune_proportion", "architecture"])


In [18]:
latex_table = create_latex_table(
        clean_table.loc[["taskInference"]],
        caption=f"{DISPLAY_NAME} with Different Hyper-Parameters \\\\ Mean Final Accuracy $\pm$ One Standard Deviation after 10 runs",
        label="tab:hp_ci_packnet",
)
copy_latex(latex_table)



/tmp/ipykernel_1015861/255770649.py:33: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(columns=["strategy", "replay_buffer", "prune_proportion", "architecture"])
